In [1]:
import os
from dotenv import load_dotenv, find_dotenv

import torch
import numpy as np
import pandas as pd

from unidecode import unidecode

import openai

In [2]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ.get("OPENAI_API_KEY")

llm_model = "gpt-3.5-turbo-0301"

In [44]:
embeddingModel = 'text-embedding-3-small'


def get_embedding(model: str, text: str) -> list[float]:
    
  text = text.replace("\n", " ")

  result = openai.embeddings.create(
      model = embedding_model,
      input = text,
      
    )
  return result.data[0].embedding

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [45]:
with open("resume/Rhys Jervis Resume - Data Scientist (1).txt", "r") as file:
    resume = file.read()


resume = resume.replace("\n", " ")  
# resume = unidecode(resume)

print(resume[:60])

ï»¿Irvington, NJ â™¦ rhysjervis2@gmail.com â™¦ 973-286-9351 


array([-0.02779602, -0.00985962,  0.02170466, ..., -0.01582941,
       -0.01988131, -0.01023104])

## Vector Search Functions

In [47]:
def euclidean_distance(tensor1, tensor2):

    # np.linalg.norm(a - b)
    return torch.dist(tensor1, tensor2,p=2)


def k_nearest_neighbors(query, tensor_list, k=None, metric = 'euclidean distance'):
    distances = []

    if k is None:
        k = len(tensor_list)

    if metric == 'euclidean distance':
        for i, tensor in enumerate(tensor_list):
            distances.append([i,euclidean_distance(query, tensor)])
    
    elif metric == 'cosine similarity':
        for i, tensor in enumerate(tensor_list):
            distances.append([i,cosine_similarity(query, tensor)])
        
    distances.sort(key=lambda x: x[1])
    return distances[:k]

# sklearn NearestNeighbors maybe

## Get data

In [48]:
df = pd.read_csv('data\df_all.csv')
df.head()

,jobTitle,company,location,date posted,link,description,qualifications,date pulled,min_salary,max_salary,embedding
0,Data Scientist,Zenith,"New York, NY",NaN,https://www.simplyhired.com/job/P_p1hn27u3cTsW...,Company Description\n\nZenith is the ROI Agenc...,Power BI||Microsoft Excel||Management||SAS||Re...,NaN,70000,105000,"[-0.0014290509279817343, 0.055856604129076004,..."
1,Senior Data Scientist,"Charlie Health Engineering, Product & Design","New York, NY",NaN,https://www.simplyhired.com/job/ikx9-0RLxNfIx5...,Why Charlie Health?\nYoung people across the c...,TensorFlow||Growing experience||Management||Py...,NaN,148500,228000,"[-0.0016314274398609996, 0.002888173097744584,..."
2,Machine Learning Engineer,FalconSmartIT,"New York, NY",NaN,https://www.simplyhired.com/job/OvfIVz13IToF5W...,Job Title: Machine Learning Engineer\nLocation...,TensorFlow||Oracle||Kubernetes||RESTful API||P...,NaN,144000,183000,"[-0.02468576468527317, -0.02219345234334469, 0..."
3,Data Scientist,Morgan Stanley,"New York, NY",NaN,https://www.simplyhired.com/job/zdQBTgDHV97jOa...,Data Scientist\nJob Number:\n3245996\nPOSTING ...,Ontology||TensorFlow||Oracle||Statistics||Doct...,NaN,150000,190000,"[-0.015597617253661156, 0.015471013262867928, ..."
4,Data Scientist,City of Jersey City,"Jersey City, NJ",NaN,https://www.simplyhired.com/job/QW6PGA25yHL-mA...,Job Opening:\nData Scientist - Innovation\nDep...,ArcGIS||Microsoft Word||Microsoft Excel||Micro...,NaN,60000,67000,"[-0.015759838744997978, 0.013187067583203316, ..."


## Get Embedings

In [64]:
from openai import embeddings
from typing import Union, List

embeddingModel = 'text-embedding-3-small'

def get_embedding(text: str, embeddingModel = 'text-embedding-3-small')->List[float]:
    text = text.replace("\n", " ")
    embedding = embeddings.create(
        input=text,
        model=embeddingModel
    )
    return embedding.data[0].embedding

In [78]:
resume_embedding = get_embedding(resume,embeddingModel )
df['embedding'] = df['description'].apply(get_embedding)

In [79]:
df['cosine_similarity' ] = df['embedding'].apply(lambda x: cosine_similarity(resume_embedding, x))

df.sort_values(by = ['cosine_similarity'] ,ascending=False, inplace=True)

df.head()

,jobTitle,company,location,date posted,link,description,qualifications,date pulled,min_salary,max_salary,embedding,cosine_similarity
57,"Healthcare Artificial Intelligence Specialist,...",NJII,"Newark, NJ",NaN,https://www.simplyhired.com/job/sKpFFbnC4rM-ce...,"Location\n100 summit street Newark, NJ, 07103,...",TensorFlow||CI/CD||Statistics||PyTorch||Softwa...,NaN,981000,124000,"[-0.04929764196276665, 0.0047972435131669044, ...",0.606444
49,Data Scientist - Hybrid - W2,Ateeca Inc,"Franklin Lakes, NJ",NaN,https://www.simplyhired.com/job/liX0Uu-5PKr_S0...,"""#INDEED_A""\nDescription:\nHybrid to remote 3 ...",R||SQL||AWS||Bachelor's degree||Clinical trial...,NaN,60000,67000,"[-0.0323323979973793, 0.02283361554145813, 0.0...",0.599057
111,Senior Data Analyst,Open Road Media,"New York, NY",NaN,https://www.simplyhired.com/job/M7cjK3441tWBoT...,Essential Functions\nDevise mathematical model...,Economics||Relational databases||R||Tableau||M...,2024-02-07,108306,108306,"[-0.029434848576784134, 0.045514997094869614, ...",0.597849
94,"Scientist 3: Data Scientist (behavior change, ...",US Tech Solutions Private Limited,"New Brunswick, NJ",NaN,https://www.simplyhired.com/job/sfahOgUOmhS1nb...,Duration: 12 months contract\nNotes about the ...,Ontology||Doctoral degree||Analysis skills||Do...,2024-02-06,6017,6017,"[-0.02734420634806156, 0.04271046072244644, 0....",0.593102
69,Data Science Program Manager - SFL Scientific,Deloitte,"New York, NY",NaN,https://www.simplyhired.com/job/lpGth_CXLKBGsD...,"Data Science Program Manager, Specialist Maste...",Doctoral degree||Master's degree||Machine lear...,2024-02-01,137000,173000,"[0.005339711904525757, 0.019791582599282265, 0...",0.561650
